In [208]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names

In [209]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [210]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"

In [211]:
revenue_band_path = os.path.join(jpg_path, 'revenue_band') + "/"
if not os.path.exists(revenue_band_path):
    os.makedirs(revenue_band_path)

In [212]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [213]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

model_types = ['cem',
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

policy_types = [m for m in model_types if m != 'cem']

# Create a DataFrame with decision_variable_names as the index
decision_variable_map = get_decision_variable_map()


In [214]:
policy_types

['pf', 'dlac-p', 'dlac-i', 'slac']

In [215]:
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [216]:
print_unique_resources = get_printable_resource_names(unique_resources)

In [217]:
unique_resources

['NG 2-on-1 Combined Cycle (F-Frame)',
 'NG Combustion Turbine (F-Frame)',
 'Land-Based Wind - Class 1 - Technology 1',
 'Utility PV - Class 1',
 'Utility-Scale Battery Storage - 2Hr',
 'Utility-Scale Battery Storage - 4Hr',
 'Utility-Scale Battery Storage - 6Hr',
 'Utility-Scale Battery Storage - 8Hr',
 'Utility-Scale Battery Storage - 10Hr']

In [218]:
non_bess_resources = [r for r in unique_resources if not any(x in r for x in ['Battery', 'BESS', 'Storage'])]

In [219]:
non_bess_resources

['NG 2-on-1 Combined Cycle (F-Frame)',
 'NG Combustion Turbine (F-Frame)',
 'Land-Based Wind - Class 1 - Technology 1',
 'Utility PV - Class 1']

In [ ]:
# initialize empty DataFrame to hold all outputs 
marginal_prices = pd.DataFrame(index=case_names, columns=non_bess_resources)

# loop throuch cases and calculate marginal prices for each technology
for case in case_names:
    # read generator data
    # storage = pd.read_csv(os.path.join(policies_path, case, 'resources/storage.csv'), index_col='Resource')
    thermal = pd.read_csv(os.path.join(policies_path, case, 'resources/Thermal.csv'), index_col='Resource')
    vre = pd.read_csv(os.path.join(policies_path, case, 'resources/Vre.csv'), index_col='Resource')

    # concatenate resources data
    non_bess_resources = pd.concat([thermal, vre], axis=0)

    thermal_fuel_types = thermal['Fuel']

    # read fuel data
    case_fuel_data = pd.read_csv(os.path.join(policies_path, case, 'system/Fuels_data.csv'))

    # get fuel prices of thermal_fuel_types
    fuel_prices = case_fuel_data[thermal_fuel_types.values]
    # calculate the average fuel price for each thermal resource
    fuel_prices_avg = fuel_prices.mean(axis=0)

    # for each row in non_bess_resources, calculate the marginal price
    for index, row in non_bess_resources.iterrows():
        # get the fuel type of the resource
        fuel_type = row['Fuel'] if row['Fuel'] != None else None
        if fuel_type is None or fuel_type not in fuel_prices_avg:
            # print(f"Fuel type {fuel_type} not found in fuel prices. Skipping resource {index}.")
            fuel_price = 0
        else:
            # get the fuel price for the resource
            fuel_price = fuel_prices_avg[fuel_type]

        resource_vom = row['Var_OM_Cost_per_MWh'] if 'Var_OM_Cost_per_MWh' in row else 0
        
        fuel_type_is_nan = pd.isna(row['Fuel'])
        fuel_type = row['Fuel'] if not fuel_type_is_nan else None
        fuel_price = 0 if fuel_type_is_nan or fuel_type not in fuel_prices_avg else fuel_prices_avg[fuel_type]
        heat_rate = row['Heat_Rate_MMBTU_per_MWh'] if not pd.isna(row['Heat_Rate_MMBTU_per_MWh']) else 0
        print(resource_vom, fuel_price, heat_rate)

        # calculate the marginal price
        marginal_price = resource_vom + (fuel_price * heat_rate)
        # add the marginal price to the DataFrame
        marginal_prices.at[case, index] = marginal_price


1.6275 5.28 6.363
5.205 6.0 9.717
0.1 0 0
0.1 0 0
3.255 5.28 6.363
10.41 6.0 9.717
0.1 0 0
0.1 0 0
3.255 10.56 6.363
10.41 12.0 9.717
0.1 0 0
0.1 0 0
3.255 5.28 6.363
10.41 6.0 9.717
0.1 0 0
0.1 0 0
3.255 10.56 6.363
10.41 12.0 9.717
0.1 0 0
0.1 0 0
3.255 15.840000000000002 6.363
10.41 18.0 9.717
0.1 0 0
0.1 0 0
3.255 21.12 6.363
10.41 24.0 9.717
0.1 0 0
0.1 0 0
3.255 5.28 6.363
10.41 6.0 9.717
0.1 0 0
0.1 0 0
3.255 10.56 6.363
10.41 12.0 9.717
0.1 0 0
0.1 0 0
3.255 5.28 6.363
10.41 6.0 9.717
0.1 0 0
0.1 0 0
3.255 10.56 6.363
10.41 12.0 9.717
0.1 0 0
0.1 0 0
3.255 5.28 6.363
10.41 6.0 9.717
0.1 0 0
0.1 0 0
3.255 10.56 6.363
10.41 12.0 9.717
0.1 0 0
0.1 0 0


In [221]:
marginal_prices

,NG 2-on-1 Combined Cycle (F-Frame),NG Combustion Turbine (F-Frame),Land-Based Wind - Class 1 - Technology 1,Utility PV - Class 1
Thermal_Base,35.22414,63.507,0.1,0.1
2_Hr_BESS,36.85164,68.712,0.1,0.1
2_Hr_BESS_Fuelx2,70.44828,127.014,0.1,0.1
4_Hr_BESS,36.85164,68.712,0.1,0.1
4_Hr_BESS_Fuelx2,70.44828,127.014,0.1,0.1
4_Hr_BESS_Fuelx3,104.04492,185.316,0.1,0.1
4_Hr_BESS_Fuelx4,137.64156,243.618,0.1,0.1
6_Hr_BESS,36.85164,68.712,0.1,0.1
6_Hr_BESS_Fuelx2,70.44828,127.014,0.1,0.1
8_Hr_BESS,36.85164,68.712,0.1,0.1


In [222]:
# print marginal prices to csv file
marginal_prices.to_csv(os.path.join(csv_path, 'info1_marginal_prices.csv'))